In [4]:
import pickle
import time

import numpy as np
from gep_config_parser import *
from data_wrangling import dataframe_to_dict

from primal_dual import PrimalDualTrainer
from gep_problem import GEPProblem
from gep_main import run_model as run_Gurobi
from gep_main import run_model_no_bounds as run_Gurobi_no_bounds
from get_gurobi_vars import OptValueExtractor
import sys
import torch
import itertools

import pyomo as pyo

In [34]:
class InputScaler:
    def __init__(self, X, scale_range=(0, 1)):
        self.min = X.min().item()
        self.max = X.max().item()
        self.scale_range = scale_range

    def scale(self, X):
        # Normalize the data
        scale_min, scale_max = self.scale_range
        return (X - self.min) / (self.max - self.min + 1e-8) * (scale_max - scale_min) + scale_min

    def inverse_scale(self, X_scaled):
        # Return normalized data back to original
        scale_min, scale_max = self.scale_range
        return (X_scaled - scale_min) * (self.max - self.min + 1e-8) / (scale_max - scale_min) + self.min


class SimplifiedGEPProblem(GEPProblem):

    def __init__(self, T, N, G, L, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap, sample_duration=12, shuffle=False, scale_input=False, train=0.8, valid=0.1, test=0.1):
        super().__init__(T, N, G, L, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap, sample_duration, shuffle, scale_input, train, valid, test)

        self.X, self.X_scaled = self._split_X_in_batches()
        self.rhs_eq, self.rhs_ineq = self._split_rhs_constants_in_batches()
        
        self._split_X_in_sets(train, valid, test)

        self._xdim = self.X.shape[1]
        
        print(f"Size of mu: {self.nineq}")
        print(f"Size of lambda: {self.neq}")
        print(f"Number of variables (size of y): {self._xdim}")
        print(f"Number of inputs (size of X): {self._ydim}")

    
    def _split_X_in_batches(self):
        """
        As input to the primal and dual nets, we use only the parameters that change over time (D_{n,t} and GA_{g,t})
        """
        # self.pDemand_tensor has shape [N, T]
        # self.pGenAva_tensor has shape [G, T]
        # Num Samples [B] = len(self.T) [8760] / self.sample_duration [12] = 730
        # Output should be of shape [B, N*sample_duration + G*sample_duration][730, N*12 + G*12]

        B = len(self.T) // self.sample_duration

        # Initialize scalers if not already initialized
        if not hasattr(self, 'pDemand_scaler'):
            self.pDemand_scaler = InputScaler(self.pDemand_tensor)

        # Scale the tensors before batching
        pDemand_scaled_full = self.pDemand_scaler.scale(self.pDemand_tensor)  # Scale the entire pDemand_tensor

        # Reshape tensors directly for batching
        pDemand_batched = self.pDemand_tensor.view(len(self.N), B, self.sample_duration).permute(1, 0, 2).reshape(B, -1)  # [B, N*sample_duration]

        pDemand_scaled_batched = pDemand_scaled_full.view(len(self.N), B, self.sample_duration).permute(1, 0, 2).reshape(B, -1)  # [B, N*sample_duration]

        return pDemand_batched, pDemand_scaled_batched
    
    def ineq_resid(self, X, y):
        """Assume y contains [*vGenProd, *vLineFlow, *vLossLoad, *vGenInv]

        Args:
            y (_type_): 

        Returns [eMaxProd, eLineFlow, eRampingDown, eRampingUp, eGenProdPositive, eMissedDemandPositive, eMissedDemandLeqDemand eNumPowerGenerationUnitsPositive]
        """
        vGenProd, vLineFlow, vLossLoad, vGenInv = self._split_y(y)

        # pDemand [B, N, T]
        # pGenAva [B, G, T]
        N = self.pDemand_tensor.shape[0]  # Number of nodes
        T = self.sample_duration          # Sample duration

        pDemand = X[:, :N * T].reshape(-1, N, T)  # Shape [B, N, T]

        pGenAva = self.pGenAva_tensor[:, :self.sample_duration]

        if self.scale_input:
            pDemand = self.pDemand_scaler.inverse_scale(pDemand)
            pGenAva = self.pGenAva_scaler.inverse_scale(pGenAva)

        # 3.1b
        # b = self._e_max_prod(vGenProd, vGenInv)
        b = self._e_max_prod_tensor(vGenProd, vGenInv, pGenAva)
        

        # 3.1d, 3.1e
        # d, e = self._e_lineflow(vLineFlow)
        d, e = self._e_lineflow_tensor(vLineFlow)
        # d, e = self._e_lineflow_tensor_scaled(vLineFlow)

        # 3.1f
        f = self._e_ramping_down_tensor(vGenProd, vGenInv)

        # 3.1g
        g = self._e_ramping_up_tensor(vGenProd, vGenInv)

        # 3.1h
        # h = self._e_gen_prod_positive(vGenProd)
        h = self._e_gen_prod_positive_tensor(vGenProd)

        # 3.1i
        # i = self._e_missed_demand_positive(vLossLoad)
        i = self._e_missed_demand_positive_tensor(vLossLoad)

        # 3.1j
        # j = self._e_missed_demand_leq_demand(vLossLoad)
        j = self._e_missed_demand_leq_demand_tensor(vLossLoad, pDemand)
        # j = self._e_missed_demand_leq_demand_tensor_scaled(vLossLoad, pDemand)

        # 3.1k
        # k = self._e_num_power_generation_units_positive(vGenInv)
        k = self._e_num_power_generation_units_positive_tensor(vGenInv)

        # g_x_y = torch.cat((b, d, e, h, i, j, k), dim=1)
        g_x_y = torch.cat((b, d, e, f, g, h, i, j, k), dim=1)

        return g_x_y

    def eq_resid(self, X, y):
        """Assume y contains [*vGenProd, *vLineFlow, *vLossLoad, *vGenInv]

        Args:
            y (_type_): _description_
        """
        vGenProd, vLineFlow, vLossLoad, _ = self._split_y(y)
        # pDemand, _ = self._split_x(X)
        N = self.pDemand_tensor.shape[0]  # Number of nodes
        T = self.sample_duration          # Sample duration

        pDemand = X[:, :N * T].reshape(-1, N, T)  # Shape [B, N, T]

        if self.scale_input:
            pDemand = self.pDemand_scaler.inverse_scale(pDemand)

        # 3.1c
        # h_x_y = self._e_nodebal(vGenProd, vLineFlow, vLossLoad)
        h_x_y = self._e_nodebal_tensor(vGenProd, vLineFlow, vLossLoad, pDemand)

        return h_x_y


In [35]:

CONFIG_FILE_NAME        = "config.toml"
VISUALIZATION_FILE_NAME = "visualization.toml"
SAMPLE_DURATION = 24
# SAMPLE_DURATION = 120


SCALE_FACTORS = {
    "pDemand": 1/1000,  # MW -> GW
    "pGenAva": 1,       # Don't scale
    "pVOLL": 1,         # kEUR/MWh -> mEUR/GWh
    "pWeight": 1,       # Don't scale
    "pRamping": 1,      # Don't scale
    "pInvCost": 1,      # kEUR/MW -> mEUR/GW
    "pVarCost": 1,      # kEUR/MWh -> mEUR/GWh
    "pUnitCap": 1/1000, # MW -> GW
    "pExpCap": 1/1000,  # MW -> GW
    "pImpCap": 1/1000,  # MW -> GW
}


## Step 1: parse the input data
print("Parsing the config file")

data = parse_config(CONFIG_FILE_NAME)
experiment = data["experiment"]
outputs_config = data["outputs_config"]

def scale_dict(data_dict, scale_factor):
    return {key: value * scale_factor for key, value in data_dict.items()}


def prep_data(inputs, shuffle=False, scale_input=True, train=0.8, valid=0.1, test=0.1, scale=False):
    print("Wrangling the input data")

    # Extract sets
    T = inputs["times"] # [1, 2, 3, ... 8760] ---> 8760
    G = inputs["generators"] # [('Country1', 'EnergySource1'), ...] ---> 107
    L = inputs["transmission_lines"] # [('Country1', 'Country2'), ...] ---> 44
    N = inputs["nodes"] # ['Country1', 'Country2', ...] ---> 20

    ### SET UP CUSTOM CONFIG ###
    # N = ['BEL', 'FRA', 'GER', 'NED'] # 4 nodes
    N = ['BEL', 'GER', 'NED'] # 3 nodes
    # G = [('BEL', 'SunPV'), ('FRA', 'SunPV'), ('GER', 'SunPV'), ('NED', 'SunPV')] # 4 generators
    G = [('BEL', 'SunPV'), ('GER', 'SunPV'), ('NED', 'SunPV')] # 3 generators
    # L = [('BEL', 'FRA'), ('BEL', 'GER'), ('BEL', 'NED'), ('GER', 'FRA'), ('GER', 'NED')] # 5 lines
    L = [('BEL', 'GER'), ('BEL', 'NED'), ('GER', 'NED')] # 3 lines

    # ! <Simplify>
    # Function to repeat the first 24 hours across all 8760 rows
    def repeat_availability(group):
        # Extract the first 24-hour values
        first_24 = group['Availability_pu'].iloc[:24].values
        # Repeat these values for 365 days (8760 rows total)
        repeated_values = (list(first_24) * (len(group) // 24))[:len(group)]
        group['Availability_pu'] = repeated_values
        return group

    # Apply the function to each country/technology group
    inputs["generation_availability_data"] = (
        inputs["generation_availability_data"]
        .groupby(['Country', 'Technology'], group_keys=False)
        .apply(repeat_availability)
    )
    # ! </Simplify>

    # Extract time series data
    pDemand = dataframe_to_dict(
        inputs["demand_data"],
        keys=["Country", "Time"],
        value="Demand_MW"
    )
    
    pGenAva = dataframe_to_dict(
        inputs["generation_availability_data"],
        keys=["Country", "Technology", "Time"],
        value="Availability_pu"
    )

    # Extract scalar parameters
    pVOLL = inputs["value_of_lost_load"]

    # WOP
    # Scale inversely proportional to times (T)
    pWeight = inputs["representative_period_weight"] / (SAMPLE_DURATION / 8760)

    pRamping = inputs["ramping_value"]

    # Extract generator parameters
    pInvCost = dataframe_to_dict(
        inputs["generation_data"],
        keys=["Country", "Technology"],
        value="InvCost_kEUR_MW_year"
    )

    pVarCost = dataframe_to_dict(
        inputs["generation_data"],
        keys=["Country", "Technology"],
        value="VarCost_kEUR_per_MWh"
    )

    pUnitCap = dataframe_to_dict(
        inputs["generation_data"],
        keys=["Country", "Technology"],
        value="UnitCap_MW"
    )

    # Extract line parameters
    pExpCap = dataframe_to_dict(
        inputs["transmission_lines_data"],
        keys=["CountryA", "CountryB"],
        value="ExpCap_MW"
    )

    pImpCap = dataframe_to_dict(
        inputs["transmission_lines_data"],
        keys=["CountryA", "CountryB"],
        value="ImpCap_MW"
    )

    if scale:
        pDemand = scale_dict(pDemand, SCALE_FACTORS["pDemand"])
        pGenAva = scale_dict(pGenAva, SCALE_FACTORS["pGenAva"])
        pVOLL *= SCALE_FACTORS["pVOLL"]
        pWeight *= SCALE_FACTORS["pWeight"]
        pRamping *= SCALE_FACTORS["pRamping"]
        pInvCost = scale_dict(pInvCost, SCALE_FACTORS["pInvCost"])
        pVarCost = scale_dict(pVarCost, SCALE_FACTORS["pVarCost"])
        pUnitCap = scale_dict(pUnitCap, SCALE_FACTORS["pUnitCap"])
        pExpCap = scale_dict(pExpCap, SCALE_FACTORS["pExpCap"])
        pImpCap = scale_dict(pImpCap, SCALE_FACTORS["pImpCap"])


    # We need to sort the dictionaries for changing to tensors!
    pDemand = dict(sorted(pDemand.items()))
    pGenAva = dict(sorted(pGenAva.items()))
    pInvCost = dict(sorted(pInvCost.items()))
    pVarCost = dict(sorted(pVarCost.items()))
    pUnitCap = dict(sorted(pUnitCap.items()))
    pExpCap = dict(sorted(pExpCap.items()))
    pImpCap = dict(sorted(pImpCap.items()))


    print("Creating problem instance")
    data = SimplifiedGEPProblem(T, N, G, L, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap, sample_duration=SAMPLE_DURATION, shuffle=shuffle, train=train, valid=valid, test=test, scale_input=scale_input)

    return data

def run_PDL(data, args, save_dir, optimal_objective_train, optimal_objective_val):
    # Run PDL
    print("Training the PDL")
    trainer = PrimalDualTrainer(data, args, save_dir, optimal_objective_train=optimal_objective_train, optimal_objective_val=optimal_objective_val)
    primal_net, dual_net, stats = trainer.train_PDL()

if __name__ == "__main__":
    args = {
            # "K": 2,
            "K": 10,
            # "L": 1,
            "L": 500,
            # "L": 2000,
            "tau": 0.8,
            # "rho": 0.1,
            "rho": 0.5,
            # "rho": 0.1,
            # "rho_max": 10,
            "rho_max": 5000,
            # "rho_max": 100,
            # "rho_max": sys.maxsize * 2 + 1,
            "alpha": 10,
            # "alpha": 2,
            "batch_size": 100,
            "hidden_sizes": [500, 500],
            # "hidden_sizes": [500, 500, 500, 500],
            # "hidden_size": 1000,
            "primal_lr": 1e-2,
            "dual_lr": 1e-2,
            # "primal_lr": 1e-5,
            # "dual_lr": 1e-5,
            # "decay": 0.99,
            "decay": 0.99,
            "patience": 10,
            "corrEps": 1e-4,
            "shuffle": False,
            "scale_input": False,
            # "train": 0.002, # 1 sample
            # "valid": 0.002,
            # "test": 0.996,
            # "train": 0.004, # 2 samples
            # "valid": 0.004,
            # "train": 0.8,
            # "valid": 0.1,
            # "test": 0.1
            # "train": 0.02,
            # "valid": 0.02,
            # "test": 0.96,
            "train": 0.01,
            "valid": 0.01,
            "test": 0.98,
    }

    # Train the model:
    for i, experiment_instance in enumerate(experiment["experiments"]):
        # Setup output dataframe
        df_res = pd.DataFrame(columns=["setup_time", "presolve_time", "barrier_time", "crossover_time", "restore_time", "objective_value"])

        for j in range(experiment["repeats"]):
            # Run one experiment for j repeats
            run_name = f"train:{args['train']}_rho:{args['rho']}_rhomax:{args['rho_max']}_alpha:{args['alpha']}_L:{args['alpha']}_scaled:{args['scale_input']}"
            save_dir = os.path.join('outputs', 'PDL',
                run_name + "-" + str(time.time()).replace('.', '-'))
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            with open(os.path.join(save_dir, 'args.dict'), 'wb') as f:
                pickle.dump(args, f)
            
            # Prep proble data:
            data = prep_data(experiment_instance, shuffle=args["shuffle"], scale_input=args["scale_input"], train=args["train"], valid=args["valid"], test=args["test"])

            # Run Gurobi
            # experiment_instance, t, N, G, L, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap
            # opt_objs_train = []
            # opt_vars = []
            # opt_dual_vars = []
            train_extractor = OptValueExtractor()
            for t in data.train_time_ranges:
                model, solver, time_taken = run_Gurobi_no_bounds(experiment_instance,
                           t,
                           data.N,
                           data.G,
                           data.L,
                           data.pDemand,
                           data.pGenAva,
                           data.pVOLL,
                           data.pWeight,
                           data.pRamping,
                           data.pInvCost,
                           data.pVarCost,
                           data.pUnitCap,
                           data.pExpCap,
                           data.pImpCap)
                train_extractor.extract_values(model)
            
            data.set_train_extractor(train_extractor)

            valid_extractor = OptValueExtractor()
            for t in data.val_time_ranges:
                model, solver, time_taken = run_Gurobi_no_bounds(experiment_instance,
                           t,
                           data.N,
                           data.G,
                           data.L,
                           data.pDemand,
                           data.pGenAva,
                           data.pVOLL,
                           data.pWeight,
                           data.pRamping,
                           data.pInvCost,
                           data.pVarCost,
                           data.pUnitCap,
                           data.pExpCap,
                           data.pImpCap)
                valid_extractor.extract_values(model)

            data.set_valid_extractor(valid_extractor)
            
            avg_opt_obj_train = train_extractor.get_avg_obj() 
            avg_opt_obj_val = valid_extractor.get_avg_obj()

            print(f"Avg obj train: {avg_opt_obj_train}")
            print(f"Avg obj valid: {avg_opt_obj_val}")

            # Run PDL
            run_PDL(data, args, save_dir, optimal_objective_train=avg_opt_obj_train, optimal_objective_val=avg_opt_obj_val)




Parsing the config file
Wrangling the input data


/var/folders/yn/jbnxb2792bx_bmgd675jx1fh0000gq/T/ipykernel_69641/4245111918.py:61: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  inputs["generation_availability_data"]


Creating problem instance
Size of train set: 3
Size of val set: 3
Size of test set: 359
Size of mu: 573
Size of lambda: 72
Number of variables (size of y): 219
Number of inputs (size of X): 144
Size of train set: 3
Size of val set: 3
Size of test set: 359
Size of mu: 573
Size of lambda: 72
Number of variables (size of y): 72
Number of inputs (size of X): 219
Populating the model
Adding model variables
Formulating the objective
Adding model constraints
Solving the optimization problem
Set parameter OutputFlag to value 1
Set parameter LogFile to value "outputs/Gurobi/output.txt"
Set parameter Crossover to value 1
Set parameter FeasibilityTol to value 1e-09
Set parameter QCPDual to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 22.3.0 22D49)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
FeasibilityTol  1e-09
Method  2
Crossover  1
QCPDual  1

Optimize a model with 647 rows, 22